In [54]:
import pandas as pd
import ajuste_sen as aj
import numpy as np
from scipy.optimize import curve_fit
import altair as alt

In [55]:
def graph(data: pd.DataFrame) -> alt.Chart:
    return (
        alt.Chart(data)
        .mark_line(color="blueviolet")
        .encode(alt.X("freq", title="Frequência"), y="bode")
    )

In [56]:
frequencia_rc = [46.25,92.5,185,370,740,1480,2960,5920,11840]
frequencia_rl = [2375.375,4750.75,9501.5,19003,38006,76012,152024,304048,608096]
amplitude_vg_rc = [2.5,2.5,2.5,2.49,2.48,2.46,2.46,2.45,2.46]
amplitude_vc_rc = [2.47,2.45,2.37,2.16,1.68,1.05,0.58,0.3,0.16]
amplitude_vg_rl = [2.27,2.27,2.28,2.29,2.3,2.36,2.44,2.47,2.46]
amplitdue_vl_rl = [0.12,0.17,0.28,0.53,0.98,1.62,2.16,2.38,2.43]

In [57]:
def bode_freq(list_freq):
    data = pd.DataFrame()
    data["freq"] = list_freq
    lista_bode = []
    for freq in list_freq:
        lista_bode.append(20 * np.log10(2 * np.pi * freq))
    data["bode"] = lista_bode
    return data

graph(bode_freq(frequencia_rc)).display()
graph(bode_freq(frequencia_rl))

alt.Chart(...)

alt.Chart(...)

In [58]:
def bode_ganho(list_freq,A_vc,A_vg):
    data = pd.DataFrame()
    data["freq"] = list_freq
    lista_bode = []
    for i in range(len(list_freq)):
        lista_bode.append(list_freq[i] * np.log10(A_vc[i]/A_vg[i]))
    data["bode"] = lista_bode
    return data

graph(bode_ganho(frequencia_rc,amplitude_vc_rc,amplitude_vg_rc)).display()
graph(bode_ganho(frequencia_rl,amplitdue_vl_rl,amplitude_vg_rl))

alt.Chart(...)

alt.Chart(...)